<a href="https://colab.research.google.com/github/EcoClim/Classification_Model/blob/main/Classification_for_EcoIdentify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**EcoIdentify**
## **Classifying Recycling Material**
###Built by EcoClim Solutions

---
Garbage classification involves separating wastes according to how it's handled or processed. It's important for recycling as some materials are recyclable and others are not.


![AI Logo](https://ecoclimsolutions.files.wordpress.com/2023/12/rmcai.png?resize=219%2C219)
---

The objective of the notebook is to build a model that helps to classify recycling material, known as EcoIdentify

---
All code is licensed under MIT License. To view terms and conditions visit this [information page](https://mit-license.org/).

#**1. Dataset**

### **About The Data And Datasets:**
* The [Dataset](https://drive.google.com/file/d/1brl6_ul9a0ILv0bylr_7ggrIQIhEBAx6/view?usp=drive_link) used is an altered version of a [Dataset](https://www.kaggle.com/datasets/asdasdasasdas/garbage-classification) found in Kaggle
* The total number of images in the dataset is 2527.
* The dataset has 6 different classes.
* **The number of images in each class varies**.
  * The **_Paper_** class has 594 images
  * The  **_Cardboard_** class has 403 images
  * The  **_Plastic_** class has 482 images
  * The  **_Glass_** class has 501 images
  * The  **_Metal_** class has 410 images
  * The  **_Trash_** class has 137 images

##**1.1 Importing Dependencies And Mounting Drive**





In [ ]:
#Importing Dependencies
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

In [ ]:
#Mounting Drive
from google.colab import drive
drive.mount('/content/drive')

## **1.2 Downloading and Storing the Dataset**

In [ ]:
#Downloading Dataset
!gdown 1brl6_ul9a0ILv0bylr_7ggrIQIhEBAx6

In [ ]:
#Changing location of Dataset
!mv /content/Dataset.zip /content/drive/MyDrive/Dataset.zip

In [ ]:
#Unizipping the Datset
!unzip '/content/drive/MyDrive/Dataset.zip' -d my_data

In [ ]:
#Declaring the Dataset's path as a variable
data_dir = 'my_data/Dataset/'

##**1.3 Reviewing the Dataset**

In [ ]:
#Reviewing number of files and directories in the Dataset
total_dir = len(os.listdir(data_dir))
total_files = 0

for dirname, _, filenames in os.walk(data_dir):
    print('counting:', dirname)
    files_counter = 0
    for file in filenames:
        files_counter += 1
    total_files += files_counter
    print('total files in dir:', files_counter)

print('--------')
print('total number of files',total_files)
print('total number of directories',total_dir)

##**1.4 Adjusting The Dataset**

In [ ]:
#Creating the Training and Validation Datasets
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    seed=100
)

validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    seed=100
)

##**1.5 Validating The Dataset**

In [ ]:
#Geting class names
class_names = train_ds.class_names
print(class_names)

##**1.6 Viewing And Inspecting The Dataset**

In [ ]:
#Viewing some images from the Training Dataset
plt.figure(figsize=(16, 16))
for images, labels in train_ds.take(1):
    for i in range(12):
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
#Viewing some images from the Validation Dataset
plt.figure(figsize=(16, 16))
for images, labels in validation_ds.take(1):
    for i in range(12):
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
#Inspect the Training Dataset
train_batch = train_ds.as_numpy_iterator().next()

print('total of batches:',len(train_ds))
print('images batch shape:',train_batch[0].shape)
print('labels batch shape:',train_batch[1].shape)

In [ ]:
#Inspect the Validation Dataset
validation_batch = validation_ds.as_numpy_iterator().next()

print('total of batches:',len(validation_ds))
print('images batch shape:',validation_batch[0].shape)
print('labels batch shape:',validation_batch[1].shape)

# **2. Neural Network**

##**2.1 Instantiating ResNet50V2 for fine-tuning**

In [ ]:
#Instantiating the base model
input_shape = (256,256,3)
base_model = tf.keras.applications.ResNet50V2(include_top=False, input_shape=input_shape)

#Making the layers of the model trainable
base_model.trainable = True

##**2.2 Reviewing the Model**

In [ ]:
#Reviewing the base model architecture
base_model.summary()

In [ ]:
#Finding the tunning layer and its index
tuning_layer_name = 'conv5_block1_preact_bn'
tuning_layer = base_model.get_layer(tuning_layer_name)
tuning_index = base_model.layers.index(tuning_layer)

#Freezing all the layers before the tuning layer
for layer in base_model.layers[:tuning_index]:
    layer.trainable =  False

##**2.3 Changing Orientation of Images for Better Precision**

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./127.5, offset=-1),
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2),
])

##**2.4 Creating the architecture**

In [ ]:
#Creating the neural network architecture
import tensorflow as tf
model = tf.keras.Sequential([
    data_augmentation,
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(6, activation='softmax')
])

learning_rate = 0.00001
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate),
    metrics=['accuracy']
)

##**2.5 Training the Model**

In [ ]:
#Training the model
history = model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=100
)

In [ ]:
#Visualizing the training history
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

#Plotting accuracy
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

#Plotting loss
plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

# **3. Model Evaluation**

##**3.1 Evaluating the Model**

In [ ]:
#Verifying the performance of the model
loss, accuracy = model.evaluate(validation_ds)
print('Test accuracy :', accuracy)
print('Test loss:', loss)

##**3.2 Saving the Model**

In [ ]:
model.save('/content/drive/MyDrive/EcoIdentify_classification_model.h5')

As of **December 23, 2025**, a newer version of Tensorflow has been released, which creates version mismatch and produces an error on deployment. To overcome this issue, run the following code and convert the model.

In [ ]:
model = tf.keras.models.load_model("/content/drive/MyDrive/EcoIdentify_classification_model.h5")
model.export("/content/drive/MyDrive/EcoIdentify_savedmodel")

# **4. Inferencing**

## **4.1. Inference Using Known Data**

In [ ]:
#Get a batch from Validation Dataset to do some inferencing
image_batch, label_batch = validation_ds.as_numpy_iterator().next()

#Inference
inference = model.predict_on_batch(image_batch)

#Viewing images and labels
plt.figure(figsize=(18, 18))
for i in range(12):
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(image_batch[i].astype("uint8"))
    plt.title('Inference:{}, {:.2f}% Confidence\nReal Label:{}'
              .format(class_names[np.argmax(inference[i])], 100 * np.max(inference[i]), class_names[label_batch[i]]))
    plt.axis("off")

#Ending Notes

The model should've saved a file named *EcoIdentify_savedmodel* in my drive. Refer to [deployment](https://huggingface.co/spaces/EcoClim-Solution/EcoIdentify) to learn how to use this file.

Similar model was also created using Roboflow. To view this, visit our Roboflow deployment at [Streamlit](https://ecoidentifyv3.streamlit.app/).